In [16]:
import altair as alt
import pandas as pd
from altair_saver import save

In [17]:
df = pd.read_csv("MusicData.csv", parse_dates=["Year"], thousands=',')
df

,﻿Format,Metric,Year,Value,Category
0,CD,Units,1973-01-01,NaN,Physical
1,CD,Units,1974-01-01,NaN,Physical
2,CD,Units,1975-01-01,NaN,Physical
3,CD,Units,1976-01-01,NaN,Physical
4,CD,Units,1977-01-01,NaN,Physical
...,...,...,...,...,...
2088,Vinyl Single,Value (Adjusted),2015-01-01,6.205390,Physical
2089,Vinyl Single,Value (Adjusted),2016-01-01,5.198931,Physical
2090,Vinyl Single,Value (Adjusted),2017-01-01,6.339678,Physical
2091,Vinyl Single,Value (Adjusted),2018-01-01,5.386197,Physical


In [19]:
# Convert Value column to float so it can be used in Altair as quantitative measure
df.astype({"Value": "float"})
df["Value"].dtype

dtype('float64')

In [20]:
# Check for leading or trailing whitespace in columns
df.columns

Index(['﻿Format', 'Metric', 'Year', 'Value', 'Category'], dtype='object')

In [21]:
# Get an overview of the format which have been used
df.groupby(['﻿Format']).sum()

,Value
﻿Format,
8 - Track,23640.469715
CD,305524.451086
CD Single,2446.015682
Cassette,102312.105824
Cassette Single,4177.126130
DVD Audio,84.535783
Download Album,12658.615697
Download Music Video,316.854561
Download Single,17199.061065


In [34]:
# Create a new column which assigns groups to each format. These groups will be displayed in the stream chart


def groups(row):
    if row["﻿Format"] == "8 - Track":
        val = "8 - Track"
    elif (
        row["﻿Format"] == "Cassette"
        or row["﻿Format"] == "Cassette Single"
        or row["﻿Format"] == "Other Tapes"
    ):
        val = "Tape"
    elif (
        row["﻿Format"] == "CD"
        or row["﻿Format"] == "CD Single"
        or row["﻿Format"] == "DVD Audio"
    ):
        val = "CD"
    elif (
        row["﻿Format"] == "Download Album"
        or row["﻿Format"] == "Download Music Video"
        or row["﻿Format"] == "Download Single"
        or row["﻿Format"] == "Kiosk"
    ):
        val = "Download"
    elif (
        row["﻿Format"] == "Music Video (Physical)"
        or row["﻿Format"] == "Other Digital"
        or row["﻿Format"] == "Music Video (Physical)"
        or row["﻿Format"] == "Ringtones & Ringbacks"
        or row["﻿Format"] == "SACD"
        or row["﻿Format"] == "SoundExchange Distributions"
        or row["﻿Format"] == "Synchronization"
    ):
        val = "Other"
    elif (
        row["﻿Format"] == "Limited Tier Paid Subscription"
        or row["﻿Format"] == "On-Demand Streaming (Ad-Supported)"
        or row["﻿Format"] == "Other Ad-Supported Streaming"
        or row["﻿Format"] == "Paid Subscription"
    ):
        val = "Stream"
    else:
        val = "Vinyl"
    return val

In [35]:
df["Format group"] = df.apply(groups, axis=1)
df

,﻿Format,Metric,Year,Value,Category,Format group
0,CD,Units,1973-01-01,NaN,Physical,CD
1,CD,Units,1974-01-01,NaN,Physical,CD
2,CD,Units,1975-01-01,NaN,Physical,CD
3,CD,Units,1976-01-01,NaN,Physical,CD
4,CD,Units,1977-01-01,NaN,Physical,CD
...,...,...,...,...,...,...
2088,Vinyl Single,Value (Adjusted),2015-01-01,6.205390,Physical,Vinyl
2089,Vinyl Single,Value (Adjusted),2016-01-01,5.198931,Physical,Vinyl
2090,Vinyl Single,Value (Adjusted),2017-01-01,6.339678,Physical,Vinyl
2091,Vinyl Single,Value (Adjusted),2018-01-01,5.386197,Physical,Vinyl


In [36]:
# Check if all rows have been assigned groups
df.groupby(['Format group']).sum()

,Value
Format group,
8 - Track,23640.469715
CD,308055.002552
Download,30229.333696
Other,26995.359465
Stream,29245.460913
Tape,106719.412794
Vinyl,108863.034759


In [41]:
selection = alt.selection_multi(fields=["Format group"], bind="legend")

chart = alt.Chart(df).mark_area().configure_axis(grid=False).encode(
    alt.X("yearmonth(Year):T", title="", axis=alt.Axis(format="%Y", domain=False, tickSize=0)),
    alt.Y("sum(Value):Q", stack="center", axis=None),
    alt.Color(
        "Format group:N",
        scale=alt.Scale(
            domain=[
                "8 - Track",
                "CD",
                "Vinyl",
                "Tape",
                "Other",
                "Download",
                "Stream",
            ],
            range=["#423e58", "#d3587b", "#6a3589", "#9b478b", "#ffae78", "#f78972", "#fbcd98"],
        ),
    ),
    tooltip=["Format group", "sum(Value):Q", "year(Year):T"],
    opacity=alt.condition(selection, alt.value(1), alt.value(0.2)),
).add_selection(selection)

chart

alt.Chart(...)

In [26]:
chart.save("music_industry.html")